In [2]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RNN, LSTM, RepeatVector
import numpy as np
import os
import tensorflow as tf
from encoder import CharacterTable
import roman_numerals as cnv
import random
import sklearn.model_selection
from sklearn.model_selection import train_test_split

In [3]:
model = Sequential()
model.add(LSTM(128,input_shape=(None,1),return_sequences=True)) # sequences of singlen numbers
model.add(LSTM(128))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer="adam",metrics=['mae','mse'])
num_epochs = 0
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 128)         66560     
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 198273 (774.50 KB)
Trainable params: 198273 (774.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [4]:
# helper method, converts sequence of numbers to text
def to_text(sample):
    return ''.join([idx2char[int(x)] for x in sample])
# helper method, converts text to sequence of numbers
def to_number(words):
    return np.array([char2idx[char] for char in words])

In [ ]:
DATASET_SIZE=200

samples = []
labels = []
all_words = ' '
max_len = 0
for i in range(DATASET_SIZE):
    labels.append(i)
    words = cnv.convert(i)
    samples.append(words)
    all_words += words
    if len(words)>max_len:
        max_len = len(words)

print('Max len of text',max_len)

vocab = sorted(set(all_words))
vocab_size = len(vocab)
print('vocabulary (used letters)',vocab)
print ('unique characters',vocab_size)

In [18]:
char2idx = {char:index for index, char in enumerate(vocab)}
print('char2idx:\n',char2idx)
idx2char = np.array(vocab)
print('idx2char\n',idx2char)

char2idx:
 {' ': 0, 'C': 1, 'I': 2, 'L': 3, 'V': 4, 'X': 5}
idx2char
 [' ' 'C' 'I' 'L' 'V' 'X']


In [19]:
samples_int = []
for s in samples:
    v = np.array([char2idx[char] for char in s])
    samples_int.append(v) # different sizes!
print(samples[123],' ->becomes-> ',samples_int[123])

CXXIII  ->becomes->  [1 5 5 2 2 2]


In [20]:
samples = np.zeros((DATASET_SIZE,max_len))
for i in range(len(samples_int)):
    for j in range(len(samples_int[i])):
        samples[i,j] = np.array(samples_int[i][j]) # all not used have '0' which is ' '
print('SAMPLES\n\n',samples)
print(samples.shape)

SAMPLES

 [[0. 0. 0. ... 0. 0. 0.]
 [2. 0. 0. ... 0. 0. 0.]
 [2. 2. 0. ... 0. 0. 0.]
 ...
 [1. 5. 1. ... 0. 0. 0.]
 [1. 5. 1. ... 2. 0. 0.]
 [1. 5. 1. ... 0. 0. 0.]]
(200, 9)


In [21]:
samples = np.expand_dims(samples,axis=2) #add the third dimension
labels = np.array(labels,dtype=float)

print("Sample (for 123):\n",samples[123])
print("Sample decoded",to_text(samples[123]))
print("Label (output):",labels[123])

print('samples shape',samples.shape)
print('labels shape',labels.shape)

Sample (for 123):
 [[1.]
 [5.]
 [5.]
 [2.]
 [2.]
 [2.]
 [0.]
 [0.]
 [0.]]
Sample decoded CXXIII   
Label (output): 123.0
samples shape (200, 9, 1)
labels shape (200,)


In [22]:
TRAINING_SIZE = .5
from sklearn.model_selection import train_test_split
(trainSamples, testSamples, trainLabels, testLabels) = train_test_split(samples, labels,train_size=TRAINING_SIZE, random_state=1)
print('Training samples:',len(trainSamples),' test samples',len(testSamples))

Training samples: 100  test samples 100


In [23]:
EPOCHS=100
BATCH_SIZE = int(len(trainSamples)/4)
print('Training with',len(trainSamples),'samples',EPOCHS,'epochs and batch_size=',BATCH_SIZE)
for x in range(100):
    H = model.fit(trainSamples, trainLabels, epochs=EPOCHS,verbose=0,batch_size=BATCH_SIZE)
    num_epochs += EPOCHS
    print("\n{}/10 Epochs: {} - loss={:6.3f}, loss improvement={:6.3f}".
          format(x, num_epochs,H.history['loss'][-1], H.history['loss'][0]-H.history['loss'][-1]))
    check_model()
print("Done")

Training with 100 samples 100 epochs and batch_size= 25

0/10 Epochs: 10100 - loss=16.790, loss improvement=2126.013
7/7 [==============================] - 0s 5ms/step
text => [predicted value] error=[error]
XXXII     =>  30.55 error = 1.45
XLIII     =>  44.97 error = 1.97
CXXXIV    =>  132.92 error = 1.08
CLXXXVII  =>  183.42 error = 3.58
LXXXIV    =>  84.93 error = 0.93
Mean error = 6.034471

1/10 Epochs: 10200 - loss= 3.489, loss improvement=12.569
7/7 [==============================] - 0s 7ms/step
text => [predicted value] error=[error]
IX        =>  9.28 error = 0.28
LXXXVIII  =>  88.11 error = 0.11
III       =>  2.99 error = 0.01
CLXXI     =>  171.22 error = 0.22
LV        =>  55.25 error = 0.25
Mean error = 4.7646303

2/10 Epochs: 10300 - loss= 1.853, loss improvement= 1.619
7/7 [==============================] - 0s 9ms/step
text => [predicted value] error=[error]
CXCV      =>  194.65 error = 0.35
XXXIX     =>  34.04 error = 4.96
CII       =>  100.15 error = 1.85
CXLVII    =>  1

In [47]:
import random

def check_model(verbose=0,how_many=5):
    pred = model.predict(samples)
    print('text => [predicted value] error=[error]')
    error = []
    for i in range(len(pred)):
        res = samples[i]
        error.append(abs(i-pred[i]))
        if verbose==1:
            train = ''
            if i in trainLabels: train='[T]'
            print(i,to_text(res),'=> {:.2f} error = {:.2f}'.format(pred[i,0],abs(i-pred[i,0])),train)
    if verbose<1: # if not verbose just display 'how_many' random samples
        for i in range(how_many):
            x = random.randrange(DATASET_SIZE)
            res = samples[x]
            print(to_text(res),'=>  {:.2f} error = {:.2f}'.format(pred[x,0],abs(x-pred[x,0])))
    print('Mean error =',np.mean(error))
    return np.mean(error)
check_model(1)

7/7 [==============================] - 0s 5ms/step
text => [predicted value] error=[error]
0           => -0.01 error = 0.01 [T]
1 I         => 0.99 error = 0.01 [T]
2 II        => 1.99 error = 0.01 [T]
3 III       => 2.98 error = 0.02 [T]
4 IV        => 6.59 error = 2.59 
5 V         => 20.02 error = 15.02 
6 VI        => 5.99 error = 0.01 [T]
7 VII       => 6.99 error = 0.01 [T]
8 VIII      => 7.99 error = 0.01 [T]
9 IX        => 8.97 error = 0.03 [T]
10 X         => 9.98 error = 0.02 [T]
11 XI        => 0.75 error = 10.25 
12 XII       => 6.56 error = 5.44 
13 XIII      => 10.75 error = 2.25 
14 XIV       => 33.14 error = 19.14 
15 XV        => 15.00 error = 0.00 [T]
16 XVI       => 16.35 error = 0.35 
17 XVII      => 17.87 error = 0.87 
18 XVIII     => 20.33 error = 2.33 
19 XIX       => 100.44 error = 81.44 
20 XX        => 19.97 error = 0.03 [T]
21 XXI       => 21.13 error = 0.13 [T]
22 XXII      => 21.88 error = 0.12 [T]
23 XXIII     => 22.98 error = 0.02 [T]
24 XXIV      => 24.

3.2561066

In [58]:
# Input must be 9 symbols
x = to_number('LXXI     ')
#              xxxxxxxxxxxxxxxxxxxxxxxxxxxxx
print(x)
x = np.expand_dims(x,axis=1)
x = np.expand_dims(x,axis=0)
model.predict(x)

[3 5 5 2 0 0 0 0 0]
1/1 [==============================] - 0s 31ms/step


array([[71.03675]], dtype=float32)

In [ ]:
model.save('model_words2numbers.h5')